In [1]:
import math
import torch
import torch.nn.functional as F
from torch import nn, Tensor

class FedAvgCNN(nn.Module):
    def __init__(self, in_features=1, num_classes=10, dim=1024):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_features,
                        32,
                        kernel_size=5,
                        padding=0,
                        stride=1,
                        bias=True),
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(kernel_size=(2, 2))
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32,
                        64,
                        kernel_size=5,
                        padding=0,
                        stride=1,
                        bias=True),
            nn.ReLU(inplace=True), 
            nn.MaxPool2d(kernel_size=(2, 2))
        )
        self.fc1 = nn.Sequential(
            nn.Linear(dim, 512), 
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = torch.flatten(out, 1)
        out = self.fc1(out)
        out = self.fc(out)
        return out

In [2]:
m = FedAvgCNN()
if isinstance(m,FedAvgCNN):
    print("yes")

yes


In [1]:
import torch
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is not available.


In [1]:
import numpy as np
import os
dataset = "Cifar100"
idx = 5
train_data_dir = os.path.join('../dataset', dataset, 'train/')

train_file = train_data_dir + str(idx) + '.npz'
with open(train_file, 'rb') as f:
    npzfile = np.load(f, allow_pickle=True)
    # 打印所有可用的键
    print(list(npzfile.keys()))

    # 检查 'data' 键是否存在
    if 'data' in npzfile:
        train_data = npzfile['data'].tolist()
    else:
        print("'data' key not found in the npz file.")

['data']


In [ ]:
def get_features_hook(module, input, output):
    # 这里的output就是经过该模块的中间输出
    # 你可以把它保存下来
    print("Shape of output:", output.shape) 


model = FedAvgCNN()
# 假设我们想要获取conv2层的中间表征
hook = model.conv2.register_forward_hook(get_features_hook)

In [ ]:
print(weights.size())

In [ ]:
import torch
model = torch.load("models\Cifar100\FedTrans_server_attn.pt")
emb_layer = model['emb_layer']
intra_attn_model = model['intra_attn_model']
psub_res = torch.load("psub_res.pth")

In [ ]:
print(psub_res[1].size())

In [ ]:
model = intra_attn_model
for name, param in model.named_parameters():
    print(f"{name, param.data}")

In [ ]:
x = torch.stack([emb_layer(p) for p in psub_res])

In [ ]:
print(x.size())
X_mean = torch.mean(x, dim=0)
X_std = torch.std(x, dim=0)
X_norm = (x - X_mean) / X_std
print(X_norm)


In [ ]:
w = intra_attn_model(X_norm)
print(w)

In [ ]:
print(x)

In [ ]:
w = intra_attn_model(x)

In [ ]:
q = intra_attn_model.query(x)
k = intra_attn_model.key(x)
print(q,k)

In [ ]:
import torch.nn.functional as F
cosine_similarity = F.cosine_similarity(q[0].unsqueeze(0),q[1].unsqueeze(0))
cosine_similarity

In [ ]:
from flcore.servers.serverfedtrans import Attn_Model
device = "cuda:0"
x.to(device)
attn_model = Attn_Model().to(device)
weights = attn_model(x)

In [ ]:
psub_res = torch.load("psub_res.pth")
def cosine_similarity(a, b):
    dot_product = torch.dot(a, b)
    norm_a = torch.norm(a)
    norm_b = torch.norm(b)
    return dot_product / (norm_a * norm_b)

print(cosine_similarity(psub_res[1],psub_res[0]))

In [ ]:
print(psub_res[0].size())

In [ ]:
attn_model = torch.load("models\Cifar10\FedTrans_server_attn.pt")
intra_attn_model = attn_model['intra_attn_model']

psub_res = [p.unsqueeze(0) for p in psub_res]

x = torch.cat(psub_res, dim=0)

In [ ]:
print(x.size())

In [ ]:
from flcore.servers.serverfedtrans import Attn_Model
attn_model2 = Attn_Model()

In [ ]:
for name, param in emb_layer.named_parameters():
    print(f"Layer: {name}, Size: {param.size()}")

In [ ]:
emb_layer = nn.Linear(x.size(1),intra_attn_model.query.weight.size(0))

In [ ]:
emb_list = [psub for psub in psub_res]

In [ ]:
iter_e, iter_w = res['inter_clusters_res']

In [ ]:
print(iter_e[1].size())

x = torch.cat(iter_e, dim=0).squeeze(1)

In [ ]:
print(x)

In [ ]:
import torch
from torch import nn

class Attn_Model_C(nn.Module):
    def __init__(self, emb_dim=128, attn_dim=128, num_heads=8):
        super(Attn_Model_C, self).__init__()
        self.emb_dim = emb_dim
        self.attn_dim = attn_dim
        self.query = nn.Linear(emb_dim, attn_dim)
        self.key = nn.Linear(emb_dim, attn_dim)
        #self.inter_LN = nn.LayerNorm(attn_dim)

        # 1-layer attention for simple verify

    def forward(self, x, models=None, prev_models=None):
        #x = self.inter_LN(x) 
        q = self.query(x)

        k = self.key(x)
        print("q:{}\n{}\nk:{}".format(q,"-"*5,k))
        scores = torch.matmul(q, k.transpose(-2, -1)) 
        #scores = torch.matmul(q, k.transpose(-2, -1)) / (self.attn_dim ** 0.2)
        print(scores)
        attention_weights = torch.softmax(scores, dim=-1)
        return attention_weights

In [ ]:

attn_model_1 = Attn_Model_C().to(device)
w = attn_model(x.to(device))

print(w)

In [ ]:
for c in res['intra_clusters_res']:
    if c is not None:
        c_e = c[0]
    else:
        continue
    
    x = torch.cat(c_e, dim=0).squeeze(1)
    w = attn_model_1(x.to(device))
    print(w)

In [ ]:

print(res['intra_clusters_res'][1])

In [ ]:
import torch
model = torch.load("gm_avg.pt")

In [ ]:
import copy
ps = []
param = [p.view(-1) for p in model.parameters()]
ps = torch.concat(param, dim=0)

In [ ]:
ps.size()

In [ ]:
ps[0]

## 测试attn参数backward

#### 开始

In [ ]:
import torch
from torch import nn

model1 = torch.nn.Linear(10, 20)
model2 = torch.nn.Linear(10, 20)
emb_layer = torch.nn.Linear(220, 128)
for p_1, p_2 in zip(model1.parameters(), model2.parameters()):
    p_1.data += p_2.data
loss = nn.MSELoss()

In [ ]:
from collections import OrderedDict

model1 = torch.nn.Linear(10, 20)
model2 = torch.nn.Linear(10, 20)
sd1 = model1.state_dict()
sd2 = model2.state_dict()
sd3 = OrderedDict()
for name, param in model1.named_parameters():
    sd3[name] = param.data.clone()
print("sd1:{}\nsd3:{}".format(sd1,sd3))
#for name, param in model1.named_parameters():
#    sd2[name] = param.data


In [ ]:
class Attn_Model(nn.Module):
    def __init__(self, emb_dim=128, attn_dim=128, num_heads=8):
        super(Attn_Model, self).__init__()
        self.emb_dim = emb_dim
        self.attn_dim = attn_dim
        self.query = nn.Linear(emb_dim, attn_dim)
        self.key = nn.Linear(emb_dim, attn_dim)
        #self.inter_LN = nn.LayerNorm(attn_dim)

        # 1-layer attention for simple verify

    def forward(self, x, models=None, prev_models=None):
        #x = self.inter_LN(x) 
        q = self.query(x)
        k = self.key(x)

        scores = torch.matmul(q, k.transpose(-2, -1)) 

        #scaled coef removed since we want to diff weight matrix entries
        #scores = torch.matmul(q, k.transpose(-2, -1)) / (self.attn_dim ** 0.5)
        attention_weights = torch.softmax(scores, dim=-1)
        return attention_weights

In [ ]:

import copy

##
def w_add_params(global_model, m_weights, models_params, require_grad=False):
    params = [] 
    res = copy.deepcopy(global_model)
    for param in res.parameters():
        param.data.zero_()
            
    for w, model_params in zip(m_weights, models_params):
        for res_param, model_param in zip(res.parameters(), model_params):
            if require_grad:
                res_param.grad += model_param.grad.clone().detach() * w
            res_param.data += model_param.data.clone().detach() * w
    return res
        
        

def emb(phead,emb_layer):
    params = []
    for p in phead.parameters():
        params.append(p.flatten())
    params = torch.cat(params)
    return emb_layer(params)


def weight_flatten(model):
    params = []
    for u in model.parameters():
        params.append(u.view(-1))
    params = torch.cat(params)

    return params

### 创建模型以及计算weighted sum

In [ ]:
models = [nn.Linear(10,20) for i in range(10)]
flatten_models = [weight_flatten(model) for model in models]
emb_layer = nn.Linear(len(flatten_models[0]), 128)
models_emb = [emb(model, emb_layer) for model in models]


In [ ]:
x = torch.cat([e.reshape(1,-1) for e in models_emb], dim=0)
print(x.size())

In [ ]:
attn_model = Attn_Model()

weights = attn_model(x)
print(weights)

#### 暂时关闭

In [ ]:
new_ms = []
for i in range(weights.size()[0]):
    w = [weights[i][j] for j in range(weights[i].size()[0])]
    sd = copy.deepcopy(models[i].state_dict())
    new_m = w_add_parameters(models[0], w, [model for model in models])
    new_ms.append(new_m)
print(len(new_ms))

In [ ]:
vecs = nn.utils.parameters_to_vector(models[0].parameters())


### 更新模型

#### 暂时关闭

In [ ]:
from collections import defaultdict, OrderedDict

In [ ]:

models = new_ms
inner_optimizers = [torch.optim.SGD(models[i].parameters(),\
                                lr=0.005) for i in range(10)]
inner_states = [copy.deepcopy(m.state_dict()) for m in models]

for i in range(10):
    #one-step local training
    inp = torch.rand([10])
    y = torch.rand([20])
    mseloss = torch.nn.MSELoss()
    #print("prev_model:{}".format(nn.utils.parameters_to_vector(models[i].parameters())))
    output = models[i](inp)
    loss = mseloss(output, y)
    loss.backward()
    inner_optimizers[i].step()
    #print("cur_model:{}".format(nn.utils.parameters_to_vector(models[0].parameters())))
final_states = [m.state_dict() for m in models]
delta_thetas = [OrderedDict({k: inner_states[i][k] - final_states[i][k] for k in models[i].state_dict().keys()}) for i in range(10)]

#### 测试模型参数修改函数

In [ ]:
from collections import OrderedDict
model = models[0]
# 我想修改的模型state
sd = model.state_dict()
w = weights[0]
# 用一个带grad的weight去aggregate模型参数
# 把参数放回state，保存带梯度的参数到另外一个list，用于梯度计算
print(sd)

def w_add_parameters(sd, w, models):
    sg = OrderedDict()
    for w_i, model in zip(w, models):
        for key in sd.keys():
            if key not in sg.keys():
                sg[key] = w_i * model.state_dict()[key]
            else:
                sg[key] = sg[key] + w_i * model.state_dict()[key]
            print(sg)
            sd[key] = sd[key] + sg[key].data
    return sg, sd
sg, sd = w_add_parameters(sd, w, models)



#### 更新模型

In [ ]:

#model = models[0]
## 我想修改的模型state
#sd = model.state_dict()
#w = weights[0]
## 用一个带grad的weight去aggregate模型参数
## 把参数放回state，保存带梯度的参数到另外一个list，用于梯度计算

#sg, sd = w_add_parameters(sd, w, models)

inner_state = sd
nm = nn.Linear(10,20)
nm.load_state_dict(sd)
inner_optimizer = torch.optim.SGD(nm.parameters(),lr=0.05)
inner_state = copy.deepcopy(nm.state_dict())
for _ in range(10):
    inp = torch.rand([10])
    y = torch.rand([20])
    mseloss = torch.nn.MSELoss()
    #print("prev_model:{}".format(nn.utils.parameters_to_vector(models[i].parameters())))
    output = nm(inp)
    loss = mseloss(output, y)
    loss.backward()
    inner_optimizer.step()
final_state = nm.state_dict()
delta_theta = OrderedDict({k:inner_state[k]-final_state[k] for k in nm.state_dict().keys()})

lv = list(sg.values())
param_list = list(attn_model.parameters())
param_list.extend(emb_layer.parameters())
params_grads = torch.autograd.grad(lv,param_list,grad_outputs=list(delta_theta.values()),retain_graph=True)

In [ ]:
at_state = copy.deepcopy(attn_model.state_dict())
emb_state = copy.deepcopy(emb_layer.state_dict())

In [ ]:

at_state_1 = copy.deepcopy(attn_model.state_dict())
emb_state_1 = copy.deepcopy(emb_layer.state_dict())

In [ ]:
lr = 0.05
optimizer = torch.optim.SGD(
            [
                {'params': [p for p in param_list]},
            ], lr=lr, momentum=0.9
        )
for p, g in zip(param_list, params_grads):
            p.grad = g
torch.nn.utils.clip_grad_norm_(param_list, 50)

optimizer.step()

In [ ]:
for p in param_list:
    print(p)

In [ ]:
inner_optimizer = torch.optim.SGD(nm.parameters(),lr=0.05)
inner_state = copy.deepcopy(nm.state_dict())
print(inner_state)
torch.autograd.set_detect_anomaly(True)
for _ in range(10):
    inp = torch.rand([10])
    y = torch.rand([20])
    mseloss = torch.nn.MSELoss()
    #print("prev_model:{}".format(nn.utils.parameters_to_vector(models[i].parameters())))
    output = nm(inp)
    loss = mseloss(output, y)
    loss.backward()
    inner_optimizer.step()
final_state = nm.state_dict()
delta_theta = OrderedDict({k:inner_state[k]-final_state[k] for k in nm.state_dict().keys()})

In [ ]:
print(inner_state)

### 计算JVP


In [ ]:
lr = 0.01
attn_optimizer = torch.optim.SGD(
    [
        {'params' : [p for p in attn_model.parameters()]} ,
        {'params' : [p for p in emb_layer.parameters()]}
    ], lr=lr, momentum=0.9)

param_list = list(attn_model.parameters())
param_list.extend(emb_layer.parameters())
attn_optimizer.zero_grad()

In [ ]:
grads = [
            torch.autograd.grad([p for p in model.parameters()], param_list , grad_outputs=list(delta_theta.values()))\
            for model, delta_theta in zip(models,delta_thetas)
        ]      
for grad in grads:
    for param, g in zip(param_list, grad):
        param.grad += grad 

torch.nn.utils.clip_grad_norm_(param_list, 50)
attn_optimizer.step()

In [ ]:
inner_states = model[0]
final_states = .state_dict()

# calculating delta theta
delta_theta = OrderedDict({k: inner_state[k] - final_state[k] for k in weights.keys()})

# calculating phi gradient
hnet_grads = torch.autograd.grad(
    list(weights.values()), hnet.parameters(), grad_outputs=list(delta_theta.values())
)

print(torch.cat(g_1,dim=0).reshape(-1,1))

In [ ]:
print(models[0].state_dict().values())

In [ ]:
mo = nn.Linear(10,20)
mo.load_state_dict(new_ms[0].state_dict())
print(mo)

In [ ]:
for p in mo.parameters():
    print(p)

In [ ]:
import random
import copy
res = copy.deepcopy(model1)
for param in res.parameters():
    param.data.zero_()

for rp, p in zip(res.parameters(), model1.parameters()):
    rp.data += p.data.clone()
    

print(res)

In [ ]:

res.clone().detach().requires_grad_(True)

In [ ]:
stri = "../data\\mnist"
stri = stri.replace("\\","/")
print(stri)

In [ ]:
import torch

torch.optim.Adam()

In [ ]:
import wandb
api = wandb.Api()
run = api.run("gerid/fedtrans_exp/t1zr3o00")
max = 0
std = 0
for i, row in run.history().iterrows():
    if max < row["test_acc"]:
        max = row["test_acc"]
        std = row["std_acc"]
         

In [ ]:
print(max,std)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.num_heads = num_heads
        self.attention = nn.MultiheadAttention(d_model, num_heads)
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        Q = self.q_linear(query)
        K = self.k_linear(key)
        V = self.v_linear(value)
        out, _ = self.attention(Q, K, V, attn_mask=mask)
        out = self.fc_out(out)
        return out

class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForwardNetwork, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForwardNetwork(d_model, d_ff)

    def forward(self, x, mask=None):
        attn_output = self.attention(x, x, x, mask)
        x = x + attn_output
        ff_output = self.feed_forward(x)
        x = x + ff_output
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff):
        super(DecoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForwardNetwork(d_model, d_ff)

    def forward(self, x, enc_out, mask=None):
        attn_output = self.attention(x, x, x, mask)
        x = x + attn_output
        attn_output = self.attention(x, enc_out, enc_out)
        x = x + attn_output
        ff_output = self.feed_forward(x)
        x = x + ff_output
        return x

class Transformer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_encoder_layers, num_decoder_layers):
        super(Transformer, self).__init__()
        self.encoder = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff) for _ in range(num_encoder_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff) for _ in range(num_decoder_layers)])

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        for layer in self.encoder:
            src = layer(src, src_mask)
        for layer in self.decoder:
            tgt = layer(tgt, src, tgt_mask)
        return tgt

# Example usage:
d_model = 512
num_heads = 8
d_ff = 2048
num_encoder_layers = 3
num_decoder_layers = 3

transformer = Transformer(d_model, num_heads, d_ff, num_encoder_layers, num_decoder_layers)

# Dummy data
src = torch.rand((10, 32, d_model))  # (seq_len, batch, d_model)
tgt = torch.rand((20, 32, d_model))  # (seq_len, batch, d_model)

out = transformer(src, tgt)
print(out.shape)  # Should be torch.Size([20, 32, 512])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# 生成100个原始数据点
x = np.linspace(0, 10, 100)
y = np.sin(x)

# 初始化插值后的数据点
x_new = np.linspace(0, 10, 1000)
y_new = []

# 对每一对相邻的原始数据点进行随机插值
for i in range(len(x) - 1):
    x_range = x[i:i + 2]
    y_range = y[i:i + 2]

    # 在只有两个点的情况下，只使用线性插值
    method = 'linear'
    
    f = interp1d(x_range, y_range, kind=method)
    x_interp = np.linspace(x_range[0], x_range[1], 10)  # 在每对相邻点之间生成10个新点
    y_interp = f(x_interp)
    
    y_new.extend(y_interp)

# 画图
plt.figure()
plt.scatter(x, y, label='Original Points')
plt.plot(x_new, y_new, label='Interpolated Points')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# 生成100个原始数据点
x = np.linspace(0, 10, 10)
y = np.sin(x)

# 初始化插值后的数据点
x_new = np.linspace(0, 10, 1000)
y_new = []

# 对每一对相邻的原始数据点进行随机插值
for i in range(len(x) - 1):
    x_range = x[i:i + 2]
    y_range = y[i:i + 2]

    # 在只有两个点的情况下，只使用线性插值
    method = 'linear'
    
    f = interp1d(x_range, y_range, kind=method)
    x_interp = np.linspace(x_range[0], x_range[1], 10)  # 在每对相邻点之间生成10个新点
    y_interp = f(x_interp)
    
    # 添加随机噪声
    noise = 0.1 * np.random.normal(size=len(y_interp))
    y_interp += noise
    
    y_new.extend(y_interp)

# 画图
plt.figure()
plt.scatter(x, y, label='Original Points')
plt.plot(x_new, y_new, label='Interpolated Points')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# 生成10个原始数据点
x = np.linspace(0, 10, 10)
y = np.sin(x)

# 初始化插值后的数据点
x_new = np.linspace(0, 10, 1000)
y_new = []

# 对每一对相邻的原始数据点进行随机插值
for i in range(len(x) - 1):
    x_range = x[i:i + 2]
    y_range = y[i:i + 2]

    # 在只有两个点的情况下，只使用线性插值
    method = 'linear'
    
    f = interp1d(x_range, y_range, kind=method)
    x_interp = np.linspace(x_range[0], x_range[1], 100)  # 在每对相邻点之间生成100个新点
    y_interp = f(x_interp)
    
    # 添加随机噪声
    noise = 0.1 * np.random.normal(size=len(y_interp))
    y_interp += noise
    
    y_new.extend(y_interp)

# 画图
plt.figure()
plt.scatter(x, y, label='Original Points', c='red')
plt.plot(x_new, y_new, label='Interpolated Points')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# 生成10个原始数据点
x = np.linspace(0, 10, 10)
y = np.sin(x)

# 初始化插值后的数据点列表
y_new = []

# 对每一对相邻的原始数据点进行随机插值
for i in range(len(x) - 1):
    x_range = x[i:i + 2]
    y_range = y[i:i + 2]

    # 使用线性插值
    method = 'linear'
    
    f = interp1d(x_range, y_range, kind=method)
    x_interp = np.linspace(x_range[0], x_range[1], 111)  # 在每对相邻点之间生成111个新点
    y_interp = f(x_interp)
    
    # 添加随机噪声
    noise = 0.1 * np.random.normal(size=len(y_interp))
    y_interp += noise
    
    # 除了最后一次迭代，去掉每次插值的最后一个点以避免重复
    if i < len(x) - 2:
        y_new.extend(y_interp[:-1])
    else:
        y_new.extend(y_interp)

# 创建新的x轴数据点
x_new = np.linspace(0, 10, len(y_new))

# 画图
plt.figure()
plt.scatter(x, y, label='Original Points', c='red')
plt.plot(x_new, y_new, label='Interpolated Points')
plt.legend()
plt.show()
